In [39]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats


import altair as alt
import hdbscan

import umap

from  itertools import *



In [2]:
datasets = [
    'boston', 
    'breastcancercoimbra',
    'breastcancerwisconsinprognostic',
    'covertype',
    'dermatology',
    'drybean',
    'echocardiogram',
    'ecoli',
    'extyaleb',
    'glassidentification',
    'heartdisease',
    'hepatitis',
    'housing', 'iris', 'mnist64', 'olive', 'weather', 'wine', 'world12d']

## 0503 Metric binning

In [292]:
metrics[['DTM_KL01', 'Trustworthiness','Cohesiveness' ]].to_json('./metric_some.json', orient='index')

In [6]:
metrics = pd.read_json('./metric_all.json', orient='index')
metrics.head()

,RMSE,DTM_KL1,DTM_KL01,DTM_KL001,Sammon,Trustworthiness,Continuity,Spearman,Steadiness,Cohesiveness
world12d_0,3953.422363,0.003898,0.024877,0.053075,0.918514,0.964815,0.991759,0.807931,0.947093,0.913462
world12d_1,3960.921143,0.004483,0.023187,0.059423,0.935628,0.959149,0.990833,0.767015,0.949786,0.909584
world12d_2,3965.792969,0.004917,0.021634,0.060510,0.949452,0.959636,0.990171,0.751908,0.915189,0.893664
world12d_3,3963.496338,0.004523,0.022761,0.063225,0.942423,0.955651,0.990852,0.769675,0.954423,0.892510
world12d_4,3958.509766,0.004037,0.021537,0.061063,0.931516,0.961658,0.992047,0.792585,0.947122,0.911067


In [114]:
scalar = StandardScaler()
z_value = pd.DataFrame(scalar.fit_transform(metrics), columns=metrics.columns, index=metrics.index).round(5)
pval = pd.DataFrame(stats.norm.cdf(z_value), index=z_value.index, columns=z_value.columns)

In [153]:
binned = pd.DataFrame(index=metrics.index)

In [158]:
key = ['Cohesiveness', 'Trustworthiness', 'DTM_KL01']
for k in key:
    binned[k] = pval[k]

In [195]:
binned_pval['T_bin']

world12d_0    8
world12d_1    8
world12d_2    8
world12d_3    8
world12d_4    8
             ..
olive_65      2
olive_66      1
olive_67      1
olive_68      0
olive_69      1
Name: T_bin, Length: 1310, dtype: category
Categories (10, object): ['0' < '1' < '2' < '3' ... '6' < '7' < '8' < '9']

In [295]:
binned_pval[['bin']]

,Trustworthiness,Cohesiveness,DTM_KL01,T_bin,C_bin,K_bin,bin
world12d_0,0.625451,0.793279,0.492879,"(0.56, 0.63]",9,4,894
world12d_1,0.612825,0.788628,0.483477,"(0.56, 0.63]",9,4,894
world12d_2,0.613916,0.768900,0.474847,"(0.56, 0.63]",8,4,884
world12d_3,0.604969,0.767430,0.481109,"(0.56, 0.63]",8,4,884
world12d_4,0.618430,0.790414,0.474310,"(0.56, 0.63]",9,4,894
...,...,...,...,...,...,...,...
olive_65,0.210035,0.108945,0.593198,"(0.14, 0.21]",1,5,215
olive_66,0.093184,0.050625,0.563496,"(0.07, 0.14]",0,5,105
olive_67,0.093184,0.047605,0.563496,"(0.07, 0.14]",0,5,105
olive_68,0.044295,0.034833,0.442344,"(-0.0007, 0.07]",0,4,004


In [196]:
binned_pval['bin'] = binned_pval['T_bin'].astype(str) + binned_pval['C_bin'].astype(str) + binned_pval['K_bin'].astype(str)

In [340]:
all_bin

,outlying,convex,skinny,skewed,clumpy,striated,sparse
world12d_0,False,True,False,False,False,False,True
world12d_1,False,True,False,False,True,False,True
world12d_2,False,True,False,False,True,True,True
world12d_3,False,True,False,False,False,True,True
world12d_4,False,True,False,False,True,True,True
...,...,...,...,...,...,...,...
olive_65,True,True,False,True,False,False,False
olive_66,True,False,True,True,False,True,False
olive_67,True,False,True,True,False,True,False
olive_68,True,False,True,True,True,True,False


In [350]:
# all_bin = pd.DataFrame(columns=['outlying', 'convex', 'skinny', 'skewed', 'clumpy', 'striated',
#        'sparse'], index=binned_pval.index)
all_bin = pd.DataFrame(index=binned_pval.index)
all_bin = all_bin.merge(binned_pval[['T_bin', 'C_bin', 'K_bin']], left_index=True, right_index=True)

In [322]:
scag = pd.read_json('./scag_bin.json')
scag.columns

Index(['outlying', 'convex', 'skinny', 'skewed', 'clumpy', 'striated',
       'sparse', 'mdp', 'cnt'],
      dtype='object')

In [352]:
all_bin.to_json('./all_bin.json', orient='index')

In [343]:
for idx, row in scag.iterrows():
    if (row.mdp):
        for m in row.mdp.split('/')[:-1]:
            all_bin.loc[m]['outlying'] = row.outlying
            all_bin.loc[m]['convex'] = row.convex
            all_bin.loc[m]['skinny'] = row.skinny
            all_bin.loc[m]['skewed'] = row.skewed
            all_bin.loc[m]['clumpy'] = row.clumpy
            all_bin.loc[m]['striated'] = row.striated
            all_bin.loc[m]['sparse'] = row['sparse']

all_bin.to_json('./all_bin.json', orient='index')

<ipython-input-343-0be979ec6476>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_bin.loc[m]['outlying'] = row.outlying
<ipython-input-343-0be979ec6476>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_bin.loc[m]['convex'] = row.convex
<ipython-input-343-0be979ec6476>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_bin.loc[m]['skinny'] = row.skinny
<ipython-input-343-0be979ec6476>:7: SettingWithCopyWarning: 
A value is try

In [232]:
binned_pval.bin.value_counts().to_csv('./binned_count.csv')

In [227]:
results = pd.DataFrame(columns=['T', 'C', 'K', 'bin',  'mdps'])

for bin, data in binned_pval.groupby(['T_bin', 'C_bin', 'K_bin', 'bin']):
    str=""
    for i in data.index:
        str += i + '/'
    results = results.append({'T': bin[0], 'C': bin[1], 'K': bin[2], 'bin': bin[3], 'mdps': str}, ignore_index=True)




,T,C,K,bin,mdps
0,0,0,0,000,glassidentification_65/glassidentification_66/...
1,0,0,4,004,breastcancerwisconsinprognostic_22/breastcance...
2,0,0,5,005,world12d_21/world12d_25/iris_23/breastcancerwi...
3,0,0,6,006,world12d_20/ecoli_46/breastcancerwisconsinprog...
4,0,0,7,007,wine_20/wine_21/wine_22/wine_23/wine_24/boston...
...,...,...,...,...,...
219,9,9,4,994,world12d_26/world12d_34/iris_19/iris_33/breast...
220,9,9,5,995,world12d_28/world12d_29/world12d_32/weather_2/...
221,9,9,6,996,world12d_10/world12d_13/weather_28/weather_33/...
222,9,9,7,997,weather_29/


In [235]:
results.to_json('./metric_binned.json', orient='records')

In [274]:
C_bins.round(3)

array([-0.001,  0.086,  0.173,  0.259,  0.346,  0.432,  0.519,  0.605,
        0.691,  0.778,  0.864])

In [282]:
stats.norm.pdf(0.95)

0.254059056469189

In [286]:
K_bins.round(2)

array([-0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [326]:
(binned_pval['T_bin'], T_bins) = pd.cut(pval['Trustworthiness'], bins=10, retbins=True, labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
(binned_pval['C_bin'], C_bins) = pd.cut(pval['Cohesiveness'], bins=10, retbins=True, labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
(binned_pval['K_bin'], K_bins) = pd.cut(pval['DTM_KL01'], bins=10, retbins=True, labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [261]:
pd.cut(pval['DTM_KL01'], bins=10, retbins=True)

(world12d_0    (0.4, 0.5]
 world12d_1    (0.4, 0.5]
 world12d_2    (0.4, 0.5]
 world12d_3    (0.4, 0.5]
 world12d_4    (0.4, 0.5]
                  ...    
 olive_65      (0.5, 0.6]
 olive_66      (0.5, 0.6]
 olive_67      (0.5, 0.6]
 olive_68      (0.4, 0.5]
 olive_69      (0.5, 0.6]
 Name: DTM_KL01, Length: 1310, dtype: category
 Categories (10, interval[float64]): [(-0.000999, 0.0999] < (0.0999, 0.2] < (0.2, 0.3] < (0.3, 0.4] ... (0.6, 0.699] < (0.699, 0.799] < (0.799, 0.899] < (0.899, 0.999]],
 array([-0.00099927,  0.09992748,  0.19985497,  0.29978245,  0.39970993,
         0.49963741,  0.5995649 ,  0.69949238,  0.79941986,  0.89934735,
         0.99927483]))

In [50]:
ranges = []
for i in range(1, 10):
    ranges.append(stats.norm.ppf(0.1 * i))

ranges

[-1.2815515655446004,
 -0.8416212335729142,
 -0.5244005127080407,
 -0.2533471031357997,
 0.0,
 0.25334710313580006,
 0.524400512708041,
 0.8416212335729143,
 1.2815515655446004]

## Archived

In [8]:
datasets_size = {
"boston": 70,
"breastcancercoimbra": 70,
"breastcancerwisconsinprognostic": 70,
"covertype": 65,
"dermatology": 70, #XX
"drybean": 65,
"echocardiogram": 70,
"ecoli": 70,
"extyaleb": 70,
"glassidentification": 70,
"heartdisease": 70,
"hepatitis": 65,
"housing": 65,
"iris": 70,
"mnist64": 70,
"olive": 70,  ##X
"weather": 70, 
"wine": 70, 
"world12d": 70
}

In [19]:
m = pd.read_json('./metric_all.json', orient='index')
s = pd.read_json('./scag_all.json', orient='index')
sm = pd.merge(s, m, left_index=True, right_index=True)

In [20]:
median = {}
for col in sm.columns:
    median[col] = sm[col].median()

for i, row in sm.iterrows():
    for col in row.index:
        if row[col] > median[col]:
            sm.loc[i, col] = True
        else:
            sm.loc[i, col] = False

s_bool = s

for i, row in s_bool.iterrows():
    for col in row.index:
        if row[col] > median[col]:
            s_bool.loc[i, col] = True
        else:
            s_bool.loc[i, col] = False
m_bool = m

for i, row in m_bool.iterrows():
    for col in row.index:
        if row[col] > median[col]:
            m_bool.loc[i, col] = True
        else:
            m_bool.loc[i, col] = False


In [ ]:
sm.drop(['stringy', 'DTM_KL1','DTM_KL001','Spearman', 'Continuity', 'Cohesiveness'], axis=1, inplace=True)
dataset = [True, False]

printList = list(product(dataset, repeat = len(sm.columns)))

x = {}
for i in printList:
    x[i] = []

for i in sm.columns:
    print(i, end=',')
print('mdp')

for i, row in sm.iterrows():
    x[tuple(row.values)].append(row.name)
for i in x:
    for ii in i:
        print(ii, end=',')
    for ii in x[i]:
        print(ii, end='/')
    print(',', end='')
    print(len(x[i]))

In [21]:
s_bool.drop(['stringy', 'monotonic'], axis=1, inplace=True)

dataset = [True, False]

printList = list(product(dataset, repeat = len(s_bool.columns)))

x = {}
for i in printList:
    x[i] = []

for i in s_bool.columns:
    print(i, end=',')
print('mdp')

outlying,convex,skinny,skewed,clumpy,striated,sparse,mdp


In [ ]:
for i, row in s_bool.iterrows():
    x[tuple(row.values)].append(row.name)
for i in x:
    for ii in i:
        print(ii, end=',')
    for ii in x[i]:
        print(ii, end='/')
    print(',', end='')
    print(len(x[i]))

In [198]:
m_bool.drop(['DTM_KL1','DTM_KL001','Spearman', 'Continuity', 'Cohesiveness'], axis=1, inplace=True)

dataset = [True, False]


printList = list(product(dataset, repeat = len(m_bool.columns)))

x = {}
for i in printList:
    x[i] = []

for i in m_bool.columns:
    print(i, end=',')
print('mdp')

RMSE, DTM_KL01, Sammon, Trustworthiness, Steadiness, mdp


In [30]:
# df_m = pd.read_csv('./metric_bin.csv', index_col=[0, 1, 2, 3, 4],keep_default_na=False)
df_s = pd.read_csv('./scag_bin.csv', index_col=[0, 1, 2, 3, 4, 5, 6],keep_default_na=False)
# df_sm = pd.read_csv('./all_bin.csv', index_col=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],keep_default_na=False)

In [10]:
binned_result_m = {}
for i , row in df_m.sort_values(by=['cnt'],ascending=False).iterrows():
    mdps = row.mdp.split('/') 
    charts = []
    for mdp in mdps:
        if (len(mdp.split('_')) < 2):
            continue
        dataset = mdp.split('_')[0]
        numProj = mdp.split('_')[1]
        xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
        xy.columns = ['x', 'y']
        label = pd.read_json(f'./ld/{dataset}/label.json')
        label.columns = ['label']
        data = pd.merge(xy, label, left_index=True, right_index=True)
        with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                meta = json.load(f)
        
        c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'{dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
        charts.append(c)
    title0 = "1:"
    title1 = "0:"
    for idx in range(len(i)):
        sc = df_m.index.names[idx]
        x = i[idx]
        if (x):
            title0 += sc + ', '
        else:
            title1 += sc + ', '
    title = title0 + ' / ' + title1
    num_per_row = 10
    x = []
    for ii in range(int(len(mdps) / num_per_row)):
            x.append(alt.hconcat(*charts[ii*num_per_row:(ii+1)*num_per_row], spacing=3))
    xx = alt.vconcat(*x).properties(title=alt.TitleParams(text=title, anchor='middle', color='darkblue', fontSize=20), spacing=3)
    binned_result_m[(tuple(i))] = xx

In [ ]:
idxx = 10
list(binned_result_m.values())[idxx]

In [31]:
idxx = 0
list(binned_result_s.values())[idxx]

IndexError: list index out of range

In [40]:
df_s.sort_values(by=['cnt'], ascending=False).cnt.values

array([143, 139,  47,  43,  36,  34,  33,  33,  32,  31,  27,  27,  26,
        26,  23,  21,  19,  19,  18,  18,  18,  17,  16,  14,  14,  14,
        14,  13,  13,  12,  12,  12,  11,  11,  11,  11,  11,  10,  10,
         9,   9,   9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   7,
         6,   6,   6,   6,   5,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   4,   4,   4,   4,   4,   4,   4,   4,   4,   3,   3,   3,
         3,   3,   3,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [ ]:
binned_result_s = {}
for i , row in df_s.sort_values(by=['cnt'],ascending=False).iterrows():
    mdps = row.mdp.split('/') 
    charts = []
    for mdp in mdps:
        if (len(mdp.split('_')) < 2):
            continue
        dataset = mdp.split('_')[0]
        numProj = mdp.split('_')[1]
        xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
        xy.columns = ['x', 'y']
        label = pd.read_json(f'./ld/{dataset}/label.json')
        label.columns = ['label']
        data = pd.merge(xy, label, left_index=True, right_index=True)
        with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                meta = json.load(f)
        
        c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'{dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
        charts.append(c)
    title0 = "1:"
    title1 = "0:"
    for idx in range(len(i)):
        sc = df_s.index.names[idx]
        x = i[idx]
        if (x):
            title0 += sc + ', '
        else:
            title1 += sc + ', '
    title = title0 + ' / ' + title1
    num_per_row = 10
    x = []
    for ii in range(int(len(mdps) / num_per_row)):
            x.append(alt.hconcat(*charts[ii*num_per_row:(ii+1)*num_per_row], spacing=3))
    xx = alt.vconcat(*x).properties(title=alt.TitleParams(text=title, anchor='middle', color='darkblue', fontSize=20), spacing=3)
    binned_result_s[(tuple(i))] = xx

In [ ]:
binned_result_sm = {}
for i , row in df_sm.sort_values(by=['cnt'],ascending=False).iterrows():
    mdps = row.mdp.split('/') 
    charts = []
    for mdp in mdps:
        if (len(mdp.split('_')) < 2):
            continue
        dataset = mdp.split('_')[0]
        numProj = mdp.split('_')[1]
        xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
        xy.columns = ['x', 'y']
        label = pd.read_json(f'./ld/{dataset}/label.json')
        label.columns = ['label']
        data = pd.merge(xy, label, left_index=True, right_index=True)
        with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                meta = json.load(f)
        
        c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'{dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
        charts.append(c)
    title0 = "1:"
    title1 = "0:"
    for idx in range(len(i)):
        sc = df_sm.index.names[idx]
        x = i[idx]
        if (x):
            title0 += sc + ', '
        else:
            title1 += sc + ', '
    title = title0 + ' / ' + title1
    num_per_row = 10
    x = []
    for ii in range(int(len(mdps) / num_per_row)):
            x.append(alt.hconcat(*charts[ii*num_per_row:(ii+1)*num_per_row], spacing=3))
    xx = alt.vconcat(*x).properties(title=alt.TitleParams(text=title, anchor='middle', color='darkblue', fontSize=20), spacing=3)
    binned_result_sm[(tuple(i))] = xx

In [1]:
list(binned_result_s.values())[10]

NameError: name 'binned_result_s' is not defined

In [ ]:
for i, row in m_bool.iterrows():
    x[tuple(row.values)].append(row.name)
res = 0
for i in x:
    for ii in i:
        print(ii, end=',')
    for ii in x[i]:
        print(ii, end='/')
    print(',', end='')
    print(len(x[i]))

In [46]:
scagnostics = ['outlying', 'convex', 'skinny', 'stringy', 'monotonic', 'skewed',
       'clumpy', 'striated', 'sparse']
# scagnostics = ['sparse']
scags = {}
num_per_scag = 100
num_per_row = 10
for scag in scagnostics:
       high = s.sort_values(by=[scag], ascending=False).index[:num_per_scag]
       charts_high = []
       for x in high:
              dataset = x.split('_')[0]
              numProj = x.split('_')[1]
              xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
              xy.columns = ['x', 'y']
              label = pd.read_json(f'./ld/{dataset}/label.json')
              label.columns = ['label']
              data = pd.merge(xy, label, left_index=True, right_index=True)
              with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                     meta = json.load(f)
              c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'({round(s.loc[x][scag], 2)}) {dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
              charts_high.append(c)
       x = []
       for i in range(int(num_per_scag / num_per_row)):
              x.append(alt.hconcat(*charts_high[i*num_per_row:(i+1)*num_per_row], spacing=3))
       
       alt_high = alt.vconcat(*x).properties(title=alt.TitleParams(text=scag+" High 100", anchor='middle', color='darkblue', fontSize=20), spacing=3)
       
       low = s.sort_values(by=[scag], ascending=True).index[:num_per_scag]
       charts_low = []
       for x in low:
              dataset = x.split('_')[0]
              numProj = x.split('_')[1]
              xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
              xy.columns = ['x', 'y']
              label = pd.read_json(f'./ld/{dataset}/label.json')
              label.columns = ['label']
              data = pd.merge(xy, label, left_index=True, right_index=True)
              with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                     meta = json.load(f)
              c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'({round(s.loc[x][scag], 2)}) {dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
              charts_low.append(c)
       x = []
       for i in range(int(num_per_scag / num_per_row)):
              x.append(alt.hconcat(*charts_low[i*num_per_row:(i+1)*num_per_row], spacing=3))
       
       alt_low = alt.vconcat(*x).properties(title=alt.TitleParams(text=scag+" low 100", anchor='middle', color='darkblue', fontSize=20), spacing=3)
       scags[scag] = (alt_high & alt_low)


In [ ]:
for s in scagnostics:
    print(f'<li><a href="./metric_viewer/scagpage/scag_{s}.html"><span style="color:#888">{s}</span></a></li>')

In [47]:
# scagnostics = ['outlying', 'convex', 'skinny', 'stringy', 'monotonic', 'skewed',
#        'clumpy', 'striated'. 'sparse']
idx = 7
for i in range(len(scagnostics)):
    scags[scagnostics[i]].save(f'./scag_{scagnostics[i]}.html')
# scags[scagnostics[0]]

In [99]:
charts = []
for x in a:
    dataset = x.split('_')[0]
    numProj = x.split('_')[1]
    xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
    xy.columns = ['x', 'y']
    label = pd.read_json(f'./ld/{dataset}/label.json')
    label.columns = ['label']
    data = pd.merge(xy, label, left_index=True, right_index=True)
    with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
        meta = json.load(f)
    c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_point().encode(
        x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
        y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
        color=alt.Color('label:N', legend=None)
    ).properties(width=150, height=150, title=f'{dataset}-{numProj}, {meta["method"]}')
    charts.append(c)
    

In [ ]:

x = []
for i in range(int(30 / 5)):
    x.append(alt.hconcat(*charts[i*5:(i+1)*5]))
    
alt.vconcat(*x).properties(title=alt.TitleParams(text=, anchor='middle'))

In [3]:
metric_type = [
    'DTM_KL1',
    'DTM_KL01',
    'DTM_KL001',
    'RMSE',
    'Sammon',
    'Spearman',
    'Trustworthiness',
    'Continuity',
    'Steadiness',
    'Cohesiveness'
    ]

In [4]:
normalized_data = {}
scaled_data = {}
raw = {}

for dataset in datasets:
    boston = pd.read_csv(f'./metric_raw/{dataset}_metrics.csv', index_col='num')
    raw[dataset] = boston

    method = boston["method"]
    boston.drop(['method'], axis=1, inplace=True)


    scaled  = StandardScaler().fit_transform(boston)

    normalized = preprocessing.normalize(boston, axis=0)
    np_normalized = np.array(normalized)
    
    boston["method"] = method
    kmeans_result = KMeans(n_clusters=10).fit(np_normalized).predict(np_normalized)
    hdbscan_result = hdbscan.HDBSCAN(min_cluster_size=3).fit_predict(np_normalized)
    normalized_data[dataset] = np_normalized
    scaled_data[dataset] = scaled
    boston["cluster_kmeans"] = kmeans_result
    boston["cluster_hdbscan"] = hdbscan_result
    cluster_kmeans = {}
    cluster_hdbscan = {}
    for idx, data in boston.groupby('cluster_kmeans'):
        cluster_kmeans[idx] = data.index.tolist()

    for idx, data in boston.groupby('cluster_hdbscan'):
        cluster_hdbscan[idx] = data.index.tolist()

    # with open(f'hdbscan/clustering_{dataset}.json', 'w') as f:
    #     json.dump(cluster_hdbscan, f)

    # with open(f'kmeans/clustering_{dataset}.json', 'w') as f:
    #     json.dump(cluster_kmeans, f)

    # boston.to_json(f'./metric/{dataset}_metrics.json', orient='index')
    

In [82]:
reducer = umap.UMAP(n_neighbors=3, min_dist=0.1, n_components=2)

for dataset in datasets[:1]:
    res_n = reducer.fit_transform(normalized_data[dataset])
    res_s = reducer.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    charts = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title=dataset+' standard', width=200, height=200)


for dataset in datasets[1:]:
    res_n = reducer.fit_transform(normalized_data[dataset])
    res_s = reducer.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title=dataset+' standard', width=200, height=200)

    charts &= res_chart

In [ ]:
charts.title = 'UMAP(n_neighbors = 3, min_dist = 0.1, n_components=2)'
charts

In [5]:
from sklearn.manifold import TSNE

In [ ]:
chartss={}
for dataset in datasets[1:]:
    perplexity = 5

    tsne = TSNE(n_components=2, perplexity=perplexity)

    res_n = tsne.fit_transform(normalized_data[dataset])
    res_s = tsne.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    charts = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title='n /  perplexity : '+ str(perplexity), width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title='s /perplexity : '+ str(perplexity), width=200, height=200)

    for perplexity in [10, 15, 20, 30, 40, 50]:
        tsne = TSNE(n_components=2, perplexity=perplexity)

        res_n = tsne.fit_transform(normalized_data[dataset])
        res_s = tsne.fit_transform(scaled_data[dataset])

        res_n = pd.DataFrame(res_n)
        res_s = pd.DataFrame(res_s)

        res_n.columns = ['x', 'y']
        res_s.columns = ['x', 'y']
        res_n.index.name = 'num'
        res_s.index.name = 'num'

        raw_d = raw[dataset]
        res_n = raw_d.merge(res_n, left_index=True, right_index=True)
        res_s = raw_d.merge(res_s, left_index=True, right_index=True)

        res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
            x='x',
            y='y',
            color='method',
        ).properties(title='perplexity : '+ str(perplexity), width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
            x='x',
            y='y',
            color='method',
            ).properties(title='perplexity : '+ str(perplexity), width=200, height=200)

        charts &= res_chart

    charts.title = 't-SNE /' + dataset
    chartss[dataset] = charts




# for dataset in datasets[1:]:
#     res_n = tsne.fit_transform(normalized_data[dataset])
#     res_s = tsne.fit_transform(scaled_data[dataset])

#     res_n = pd.DataFrame(res_n)
#     res_s = pd.DataFrame(res_s)

#     res_n.columns = ['x', 'y']
#     res_s.columns = ['x', 'y']
#     res_n.index.name = 'num'
#     res_s.index.name = 'num'

#     raw_d = raw[dataset]
#     res_n = raw_d.merge(res_n, left_index=True, right_index=True)
#     res_s = raw_d.merge(res_s, left_index=True, right_index=True)

#     res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
#         x='x',
#         y='y',
#         color='method',
#     ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
#         x='x',
#         y='y',
#         color='method',
#         ).properties(title=dataset+' standard', width=200, height=200)

#     charts &= res_chart